In [9]:
%load_ext autoreload
%autoreload 2

In [16]:
import torchvision.models as models
from cifar_utils import get_datasets_res, plot, show_preds
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score as acc

In [17]:
def fit(net, crit, opt, train, val, n_epochs=1):
    for epoch in range(n_epochs):
        epoch_loss = []
        epoch_acc = []
        for X, y in train:
            opt.zero_grad()
            p = net(X)
            v, i = p.max(axis=1)
            epoch_acc.append(acc(y, i))
            loss = crit(p, y)
            epoch_loss.append(loss.item())
            loss.backward()
            opt.step()
        with torch.no_grad():
            for X, y in val:
                p = net(X)
                v, i = p.max(axis=1)
                val_loss = crit(p, y).item()
                val_acc = acc(y, i)
        print(f"Epoch:{epoch + 1}, T Loss:{np.mean(epoch_loss):.3f}, T acc:{np.mean(epoch_acc):.3f} V Loss:{np.mean(val_loss):.3f}, V acc:{val_acc:.3f}")

In [18]:
resnet18 = models.resnet18(pretrained=True)
for param in resnet18.parameters():
    param.requires_grad = False

In [19]:
FOLDER = ".//data//cifar//"
training_set, validation_set, mini_set = get_datasets_res(FOLDER)

In [23]:
training_gen = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=4)
valid_gen = DataLoader(validation_set, batch_size=len(validation_set))
mini_gen = DataLoader(mini_set, batch_size=32, shuffle=True, num_workers=4)

In [20]:
resnet18.fc = nn.Linear(512, 10)
crit = nn.CrossEntropyLoss()
opt = optim.Adam(resnet18.parameters())

In [24]:
fit(resnet18, crit, opt, training_gen, valid_gen, 1)

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\stuff\py\env\rl\lib\site-packages\torch\utils\data\_utils\worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\stuff\py\env\rl\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\stuff\py\env\rl\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Stuff\py\projects\ml\ml\cifar_utils.py", line 55, in __getitem__
    return self.transform(self.X[idx]), self.y[idx]
  File "c:\stuff\py\env\rl\lib\site-packages\torchvision\transforms\transforms.py", line 70, in __call__
    img = t(img)
  File "c:\stuff\py\env\rl\lib\site-packages\torchvision\transforms\transforms.py", line 207, in __call__
    return F.resize(img, self.size, self.interpolation)
  File "c:\stu